Saved as tweet_id_aggregation.ipynb in the project folder

In [ ]:
import numpy as np
import pandas as pd
import os
import sys

In [ ]:
!pip install -U pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [ ]:
# For Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path_to_utils='/content/drive/My Drive/NLP'
sys.path.append(path_to_utils)

In [ ]:
os.chdir(path_to_utils)

print(os.getcwd())

/content/drive/.shortcut-targets-by-id/1qJ8h8Q6C6t_FWn4R8pQeH813twKX9HgP/NLP


In [ ]:
# folder = os.getcwd() + '/data/'

df1 = pd.read_csv('train_reply_labels_set1.txt', delimiter = "\t", dtype = {'id': str})
df2 = pd.read_csv('train_reply_labels_set2.txt', delimiter = "\t", dtype = {'id': str})

In [ ]:
dframes = [df1, df2]

df_all_temp = pd.concat(dframes)
df_all = df_all_temp.copy()

In [ ]:
# One hot encoding for neutral, positive, negative
# for neutral
def make_neutral(x):
    val = x
    if val=="neutral":
        val = 1
    else:
        val = 0
    return val

# for positive
def make_pos(x):
    val = x
    if val=="positive":
        val = 1
    else:
        val = 0
    return val

# for negative
def make_neg(x):
    val = x
    if val=="negative":
        val = 1
    else:
        val = 0
    return val

In [ ]:
df_all['l_neutral'] = df_all['label'].apply(make_neutral)
df_all['l_pos'] = df_all['label'].apply(make_pos)
df_all['l_neg'] = df_all['label'].apply(make_neg)

In [ ]:
# df_all.head()

In [ ]:
# use sudo sql to sum by label and id
query = "SELECT id, SUM(l_neutral) AS Neutral, SUM(l_pos) AS Positive, SUM(l_neg) AS Negative FROM df_all GROUP BY id"

df_pivot = sqldf(query)
df_pivot['Total'] = df_pivot['Neutral'] + df_pivot['Positive'] + df_pivot['Negative']
# df_pivot.head()

In [ ]:
# Function to create labels

epsilon = 1e-7 #to avoid dividing by zero

def newlabel(row):
    if (row['Neutral'] / (row['Total'] + epsilon)) > 0.85:
        return 'Neutral'
    else:
        if row['Positive'] > 1.5 * row['Negative']:
            return 'Positive'
        elif row['Negative'] > 1.6 * row['Positive']:
            return 'Negative'
        else:
            return 'Neutral'

In [ ]:
df_label = df_pivot
df_label = df_label.copy()

In [ ]:
df_label['new_label'] = df_label.apply(newlabel, axis=1)
# df_label.head()

In [ ]:
# df_label.info()

In [ ]:
df_forcsv_temp = df_label
df_forcsv_temp = df_forcsv_temp.copy()

new_cols = ['id', 'new_label']

df_forcsv = df_forcsv_temp[new_cols]
df_forcsv = df_forcsv.copy()
df_forcsv

,id,new_label
0,1081277365785976832,Neutral
1,1081465965601271808,Neutral
2,1082049696904941568,Neutral
3,1083563262724186112,Neutral
4,1084152409255866368,Neutral
...,...,...
34516,1233463115611824128,Neutral
34517,1233485144247459840,Negative
34518,1233512637276807168,Negative
34519,1233514593479249921,Neutral


In [ ]:
# To export to csv
# df_forcsv.to_csv('tweet_labels_newcalc.csv', index=False)